In [1]:
import glob as g
import ujson
import os
import os.path as op
import re
import subprocess as sp
from subprocess import CalledProcessError
import time
import ocrolib
import copy

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os.path
sys.path.append(os.path.abspath('../catconv'))
import catconv.stabi as sb
import catconv.operations as cop

In [26]:
from multiprocessing.pool import Pool

pages = map(pages_from_paths, catalog_pages('../SD_png',ext='.pseg.png'))[:10]
read_page = lambda page: read_segments(page['path'])
pool = Pool(10)
pages = [page for page in pool.imap(load_page_text,pages)]

pages[7]

{'lines': [{'name': '010001', 'position': [45L, 536L, 460L, 558L]},
  {'name': '010002', 'position': [60L, 440L, 442L, 464L]},
  {'name': '010003', 'position': [59L, 392L, 148L, 413L]},
  {'name': '010004', 'position': [59L, 356L, 349L, 378L]},
  {'name': '010005', 'position': [58L, 318L, 213L, 343L]},
  {'name': '010006', 'position': [753L, 506L, 896L, 526L]}],
 'path': '../SD_png/SD012/00000146.pseg.png'}

In [4]:
crop_conv = {'cat': 'SD_png', 'ext': '.png', 'remove_type': True}
seg_conv  = {'ext': '.pseg.png'}
bin_conv  = {'cat': 'SD_png', 'ext': '.bin.png'}
croppings = map(lambda old: {'from': old, 'to': convert_path(old,crop_conv)}, db['input_files'])
pngs      = map(lambda pair: pair['to'], croppings)
segmented = map(lambda croped: convert_path(croped, seg_conv), pngs)
binarized = map(lambda croped: convert_path(croped['to'], bin_conv), croppings)

crop_jobs = map(lambda conv: crop(conv['from']['path'],conv['to']['path']), croppings)
bin_jobs  = map(lambda png: binarize(png['path']), pngs)
seg_jobs  = map(lambda png: segment( png['path']), binarized)

NameError: name 'db' is not defined

In [5]:
created_dirs   = map(lambda conv: create_dirs(conv['to']['path']), croppings)
executions     = map(execute_job, crop_jobs)
bin_executions = map(execute_job, bin_jobs)
seg_executions = map(execute_job, seg_jobs)


NameError: name 'croppings' is not defined

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.path import Path
import numpy as np

def vertices(line):
    x0, y0 ,x1,y1 = line
    return {'verts': 
            [(x0, y0),
            (x1, y0),
            (x1, y1), 
            (x0, y1),
            (x0, y0)],
            'codes':
            [Path.MOVETO,
             Path.LINETO,
             Path.LINETO,
             Path.LINETO,
             Path.CLOSEPOLY]}

def plot_lines(verts):
    return [v[0] for v in verts], [v[1] for v in verts]



In [7]:
def page_filter(page):
    return True
def all_filter(arg):
    return True
def line_filter(line):
    x0, y0 ,x1,y1 = line['position']
    if y0 < 450 and x0 > 400:
        return True
    else:
        return False


In [8]:
def line_image(page, line):
    return op.join(page_dir(page['path']), line['name'] + '.bin.png')

In [9]:
def dump_images_to_html_string(images):
    img  = """<p>{}<img src=\"{}\"/></p>"""
    html = """<html><body>{}</body></html>"""
    elements = []
    for image in images:
        elements.append(img.format(image,op.abspath(image)))
        
    return html.format('\n'.join(elements))

In [10]:
card_size_verts = vertices([0,0,1008,608])
images = []
for page in filter(page_filter, catalog['pages']):
    for line in filter(line_filter, page['lines']):
        plt.plot(*plot_lines(vertices(line['position'])['verts']))
        images.append(line_image(page,line))

plt.xlim([0,1008])
plt.ylim([0,608])
plt.title('Gefilterte Textzeilen')
plt.ylabel('Position in px')
plt.xlabel('Position in px')
plt.savefig('plots/filtered_num.png')
html = dump_images_to_html_string(images[:200:2])
with open('filtered_lines.html', 'w') as out:
    out.write(html)
    

NameError: name 'catalog' is not defined

In [11]:
CAT_NAME = 'SD'
CAT_CONV = 'SD_png'
CAT_PATH = '../SD_png/'
DB_PATH  = 'data/convert.json'
N = 10
# Example page
page1 = {'path': '../SD_png/SD012/00000111'}

In [12]:
load_catalog
# batched execution:
bin_job  = lambda page: binarize(page['path'])
seg_job  = lambda page: segment( page['path'])
bin_is_complete = lambda page: op.isfile(change_path(page['path'],ext='.bin.png'))
seg_is_complete = lambda page: op.isfile(change_path(page['path'],ext='.pseg.png'))
job_queues  = {'binarisieren': (bin_job,bin_is_complete), 'segmentieren' : (seg_job, seg_is_complete)}
for task in job_queue:
    process, fcomplete = task
    



NameError: name 'load_catalog' is not defined

In [13]:

for idx in batches(CAT_PATH):
    for name, job in job_queue.items():
        batched_pages = [{'path': path} for path in catalog_pages(CAT_PATH,batch=idx,ext='.png')]
        print('Starting execution {} with batch {} ({} images)'.format(name, idx, len(batched_pages)))
        outstanding = [page for page in batched_pages if not job_filter[name](page)]
        print 'outstanding jobs:', len(outstanding)
        jobs  = map(job,  outstanding)
#         execution = execute_queue(jobs)
#         print execution['time']
#         executions = map(execute_job, jobs)
#         execution_stats(executions)
    
        print('Done')

Starting execution binarizieren with batch SD012 (446 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch SD012 (446 images)
outstanding jobs: 5
Done
Starting execution binarizieren with batch SD006 (544 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch SD006 (544 images)
outstanding jobs: 4
Done
Starting execution binarizieren with batch SD002 (833 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch SD002 (833 images)
outstanding jobs: 4
Done
Starting execution binarizieren with batch SD004 (466 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch SD004 (466 images)
outstanding jobs: 0
Done
Starting execution binarizieren with batch SD005 (735 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch SD005 (735 images)
outstanding jobs: 3
Done
Starting execution binarizieren with batch SD003 (614 images)
outstanding jobs: 0
Done
Starting execution segmentieren with batch 

In [234]:
model = '../htwmusik-models/catalog2-gt-model.model-00101000.pyrnn.gz'
pre_job = lambda page: predict(op.join(change_path(page['path']),'*.bin.png'), model)
selected_pages = filter(seg_is_complete, catalog['pages'])
selected_pages = filter(seg_is_complete, selected_pages)
has_no_text = lambda page: op.isdir(sb.page_dir(page['path'])) and not all(map(op.isfile, text_files(page).values()))
selected_pages = filter(has_no_text, selected_pages)
print page1
print has_no_text(page1)
print len(selected_pages)
pre_jobs       = map(pre_job, selected_pages)
pre_job(selected_pages[1])
# execute_job(pre_job(selected_pages[1]))

{'path': '../SD_png/SD012/00000111', 'lines': [{'position': [86L, 528L, 874L, 557L], 'name': '010001', 'text': ''}, {'position': [47L, 478L, 226L, 501L], 'name': '010002', 'text': 'Holli er Heinz:\n'}, {'position': [47L, 439L, 913L, 466L], 'name': '010003', 'text': 'Schneewittchen : Oper in 5 Szenen, einem Prolog und einem Epilog nach\n'}, {'position': [46L, 405L, 913L, 433L], 'name': '010004', 'text': 'Robert Walser / Heinz Holliger. - Grifelfing : ECM-Records-Verlag;\n'}, {'position': [46L, 369L, 915L, 398L], 'name': '010005', 'text': 'Hamburg : Universal Music, Vertrieb, [2001]. - 2 Compact Discs in Schuber\n'}, {'position': [47L, 335L, 714L, 362L], 'name': '010006', 'text': '+ Beih. (mit dt, engl, franz Libretto). - (ECM new series)\n'}, {'position': [47L, 301L, 913L, 328L], 'name': '010007', 'text': ''}, {'position': [46L, 266L, 913L, 294L], 'name': '010008', 'text': ''}, {'position': [45L, 233L, 914L, 260L], 'name': '010009', 'text': ''}, {'position': [46L, 205L, 106L, 225L], 'na

['ocropus-rpred',
 '-Q',
 '1',
 '-m',
 '../htwmusik-models/catalog2-gt-model.model-00101000.pyrnn.gz',
 '../SD_png/SD012/00000321/*.bin.png']

In [13]:
page1

{'path': '../SD_png/SD012/00000111'}

In [14]:
# pre_execution = map(execute_job,pre_jobs)

In [15]:
selected_pages

NameError: name 'selected_pages' is not defined

In [16]:
def text_files(page):
    if 'lines' in page:
        return {line['name']: op.join(page['path'],line['name'] + '.txt') for line in page['lines']}
    else:
        return {}

In [17]:
page1

{'path': '../SD_png/SD012/00000111'}

In [ ]:
pre_excecution = map(execute_job,pre_jobs)

In [18]:
pre_job = lambda page: predict(page['path'], model)
pre_jobs       = map(pre_job, pages)
pre_excecution = map(execute_job,pre_jobs)

NameError: name 'pages' is not defined

In [230]:
execution_stats(pre_excecution)

(8.1798688471317291, 's per job')
(0, 'not completed')


In [248]:
sb.load_box_positions(page1)
sb.read_text(page1)
page1

{'lines': [{'name': '010001', 'position': [86L, 528L, 874L, 557L], 'text': ''},
  {'name': '010002',
   'position': [47L, 478L, 226L, 501L],
   'text': 'Holli er Heinz:\n'},
  {'name': '010003',
   'position': [47L, 439L, 913L, 466L],
   'text': 'Schneewittchen : Oper in 5 Szenen, einem Prolog und einem Epilog nach\n'},
  {'name': '010004',
   'position': [46L, 405L, 913L, 433L],
   'text': 'Robert Walser / Heinz Holliger. - Grifelfing : ECM-Records-Verlag;\n'},
  {'name': '010005',
   'position': [46L, 369L, 915L, 398L],
   'text': 'Hamburg : Universal Music, Vertrieb, [2001]. - 2 Compact Discs in Schuber\n'},
  {'name': '010006',
   'position': [47L, 335L, 714L, 362L],
   'text': '+ Beih. (mit dt, engl, franz Libretto). - (ECM new series)\n'},
  {'name': '010007', 'position': [47L, 301L, 913L, 328L], 'text': ''},
  {'name': '010008', 'position': [46L, 266L, 913L, 294L], 'text': ''},
  {'name': '010009', 'position': [45L, 233L, 914L, 260L], 'text': ''},
  {'name': '01000a', 'position'

In [19]:
def pretty(catalog):
    print ujson.encode(catalog,ensure_ascii=False,indent=2)

In [56]:
from gevent.pool import Pool
# from gevent. import Pool

def load_catalog(path, selection={}, load_box_positions=False, read_text=False):
    name = op.basename(path)
    pages = map(sb.pages_from_paths,sb.catalog_pages(path,**selection))

    for page in pages:
        if load_box_positions:
            sb.load_box_positions(page)
        if read_text:
            sb.read_text(page)
#     pool = Pool(7)
#     pages = [page for page in pool.imap(load_page_text,pages)]
    return {'name': name, 'path': path, 'pages': pages}

def change_paths(cat, conv):
    return [sb.convert_path(page) for page in cat['pages']]

batch1 = {'batch': 'SD012'}
catalog = load_catalog('../SD_png',selection=batch1, load_box_positions=True, read_text=True)

In [259]:
texted = [page for page in catalog['pages'] if op.basename(sb.change_path(page['path'])) == '00000111']
print has_no_text(texted[0])
print texted[0]
sb.read_text(texted[0])
texted

True
{'path': '../SD_png/SD012/00000111.png', 'lines': [{'position': [86L, 528L, 874L, 557L], 'name': '010001', 'text': ''}, {'position': [47L, 478L, 226L, 501L], 'name': '010002', 'text': 'Holli er Heinz:\n'}, {'position': [47L, 439L, 913L, 466L], 'name': '010003', 'text': 'Schneewittchen : Oper in 5 Szenen, einem Prolog und einem Epilog nach\n'}, {'position': [46L, 405L, 913L, 433L], 'name': '010004', 'text': 'Robert Walser / Heinz Holliger. - Grifelfing : ECM-Records-Verlag;\n'}, {'position': [46L, 369L, 915L, 398L], 'name': '010005', 'text': 'Hamburg : Universal Music, Vertrieb, [2001]. - 2 Compact Discs in Schuber\n'}, {'position': [47L, 335L, 714L, 362L], 'name': '010006', 'text': '+ Beih. (mit dt, engl, franz Libretto). - (ECM new series)\n'}, {'position': [47L, 301L, 913L, 328L], 'name': '010007', 'text': ''}, {'position': [46L, 266L, 913L, 294L], 'name': '010008', 'text': ''}, {'position': [45L, 233L, 914L, 260L], 'name': '010009', 'text': ''}, {'position': [46L, 205L, 106L, 2

[{'lines': [{'name': '010001',
    'position': [86L, 528L, 874L, 557L],
    'text': ''},
   {'name': '010002',
    'position': [47L, 478L, 226L, 501L],
    'text': 'Holli er Heinz:\n'},
   {'name': '010003',
    'position': [47L, 439L, 913L, 466L],
    'text': 'Schneewittchen : Oper in 5 Szenen, einem Prolog und einem Epilog nach\n'},
   {'name': '010004',
    'position': [46L, 405L, 913L, 433L],
    'text': 'Robert Walser / Heinz Holliger. - Grifelfing : ECM-Records-Verlag;\n'},
   {'name': '010005',
    'position': [46L, 369L, 915L, 398L],
    'text': 'Hamburg : Universal Music, Vertrieb, [2001]. - 2 Compact Discs in Schuber\n'},
   {'name': '010006',
    'position': [47L, 335L, 714L, 362L],
    'text': '+ Beih. (mit dt, engl, franz Libretto). - (ECM new series)\n'},
   {'name': '010007', 'position': [47L, 301L, 913L, 328L], 'text': ''},
   {'name': '010008', 'position': [46L, 266L, 913L, 294L], 'text': ''},
   {'name': '010009', 'position': [45L, 233L, 914L, 260L], 'text': ''},
   {

In [ ]:
relPagePaths = {'rel_path': '../', 'remove_type': True}
change_paths(catalog, relPagePaths)

In [11]:
change_path('../SD_png/SD012/00000282.pseg.png',rel_path='../',remove_type=True)

'SD_png/00000282.pseg'

In [12]:
op.basename('../SD_png/SD012/00000282.pseg.png')

'00000282.pseg.png'

In [63]:
import copy

export_conv = {'rel_path': '../SD_png'}
print catalog['pages'][1]['path']
c_catalog = copy.deepcopy(catalog)
limit = 2
with open('data/SD_png.json','wb') as export:
    for page in c_catalog['pages']:
        page['path'] = sb.change_path(page['path'],**export_conv)
    c_catalog['pages'] = c_catalog['pages'][:limit]
    export.write(ujson.dumps(c_catalog, ensure_ascii=False,indent=2))
print c_catalog['pages'][1]['path']

../SD_png/SD012/00000321.png
SD012/00000321


In [62]:
!cat data/SD_png.json

{
  "path":"..\/SD_png",
  "name":"SD_png",
  "pages":[
    {
      "path":"SD012\/00000327",
      "lines":[
        {
          "position":[
            22,
            529,
            484,
            564
          ],
          "name":"010001",
          "text":"Drchestre Philharmonique\n"
        },
        {
          "position":[
            85,
            503,
            299,
            525
          ],
          "name":"010002",
          "text":"dMonte Carlo>\n"
        },
        {
          "position":[
            763,
            475,
            855,
            498
          ],
          "name":"010003",
          "text":"CD 2080\n"
        },
        {
          "position":[
            32,
            406,
            877,
            431
          ],
          "name":"010004",
          "text":"Chausson Ernest-Am~dee: Poane de 1'amour et de la mer [Forts.]\n"
        },
        {
          "position":[
            32,
            333,
            918,
            

In [158]:
print sb.change_path(catalog['pages'][1]['path'],**export_conv)
print op.join(catalog['path'],catalog['pages'][1]['path'])

../catalog-ocr-test/SD012/00000321
../SD_png/SD012/00000321


In [232]:
any(map(op.isfile, text_files(page1).values()))

True

In [64]:
import itertools as it
lines = [page['lines'] for page in catalog['pages']]
# text = [line['text'] for line in lines]
all_lines = it.chain(*lines)
text = [line['text'] for line in all_lines]
